In [55]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
import re
import nltk
import string
from nltk.tokenize import TweetTokenizer
import unidecode

import warnings
warnings.filterwarnings('ignore')

stopwords = nltk.corpus.stopwords.words('english')

In [2]:
tweets = pd.read_csv("../data/train.csv")

In [4]:
def remove_punctuation(word):        
    clean_word = ''.join([char for char in word if char not in string.punctuation])
    return clean_word

In [5]:
def cleaning_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    text_tokenize = tokenizer.tokenize(text)
    wordlist = []
    for word in text_tokenize:
        word = word.lower()
        word = re.sub('(?P<url>https?://[^\s]+)', ' ', word)
        word = remove_punctuation(word)
        word = re.sub(r'[^\w]', ' ', word)
        word = unidecode.unidecode(word)
        word = re.sub(r'[0-9]','', word)
        if((word != '')&(word != ' ')&(word not in stopwords)):
            wordlist.append(word)
    clean_text = ' '.join(wordlist)
    return clean_text

In [6]:
tweets["clean_text"] = tweets["text"].apply(lambda x: cleaning_text(x))
tweets.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfires pou...


In [7]:
tweets.tail()

,id,keyword,location,text,target,clean_text
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant cranes holding bridge collapse nearb...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,control wild fires california even northern pa...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,utc km volcano hawaii
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,police investigating ebike collided car little...
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1,latest homes razed northern california wildfir...


In [8]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tweets["clean_text"])
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
feature_words = vectorizer.get_feature_names()
len(feature_words)

14350

In [10]:
df = pd.DataFrame(X.toarray(), columns=feature_words)
df.head()

,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,...,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
print(df.loc[:,(df.sum()>1)].shape)
print(df.loc[:,(df.sum()>2)].shape)
print(df.loc[:,(df.sum()>3)].shape)

(7613, 3968)
(7613, 2138)
(7613, 1484)


In [19]:
print(df.loc[:,(df.sum()>=4)].shape)
print(df.loc[:,(df.sum()>=5)].shape)

(7613, 1156)
(7613, 911)


In [20]:
print(df.loc[:,(df.sum()>6)].shape) #probar con este

(7613, 748)


In [11]:
df_filter = df.loc[:,(df.sum()>6)]
df_filter["target_label"] = tweets.target
df_filter.head(2)

,abc,ablaze,accident,across,actually,affected,aftershock,ago,aint,air,...,year,yearold,years,yes,yet,youre,youth,yr,zone,target_label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [12]:
X = df_filter.drop(["target_label"], axis=1)
y = df_filter["target_label"]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [14]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5709, 748)
(1904, 748)
(5709,)
(1904,)


#### Ajustando hiper-parametros (usando 748 feature-words):
    - n_estimators=50, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.761029
    - n_estimators=50, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.767332
    - n_estimators=100, max_depth=7, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.769958
    - n_estimators=100, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.773634
    - n_estimators=100, max_depth=15, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.773786
    - n_estimators=100, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=0.1, alpha=0
    SCORE: 0.777311
    - n_estimators=100, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.3, lambda=0.1, alpha=0
    SCORE: 0.775210
    - n_estimators=200, max_depth=9, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=0.5, alpha=0
    SCORE: 0.777836
    - n_estimators=200, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=0.1, alpha=0
    SCORE: 0.783088
    - n_estimators=200, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.7, lambda=0.1, alpha=0
    SCORE: 0.787815
    - n_estimators=200, max_depth=15, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=0.1, alpha=0
    SCORE: 0.785189
    - n_estimators=250, max_depth=13 u 11, learning_rate=0.1, subsample=1, colsample_bytree=0.7, lambda=0.1, alpha=0
    SCORE: 0.790966
    - n_estimators=250, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.7, lambda=0.1, alpha=0.1
    SCORE: 0.789391

In [50]:
model_xgb = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', max_depth=11, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.7, reg_lambda=0.1, n_jobs=1)
model_xgb.fit(X_train, y_train)

XGBClassifier(alpha=0.1, base_score=0.5, booster='gbtree',
       colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
       gamma=0, gpu_id=-1, importance_type='gain',
       interaction_constraints='', learning_rate=0.1, max_delta_step=0,
       max_depth=11, min_child_weight=1, missing=nan,
       monotone_constraints='()', n_estimators=250, n_jobs=1,
       num_parallel_tree=1, objective='binary:logistic', random_state=0,
       reg_alpha=0.100000001, reg_lambda=0.1, scale_pos_weight=1,
       subsample=1, tree_method='exact', validate_parameters=1,
       verbosity=None)

In [51]:
y_test_hat = model_xgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.789391


In [52]:
#tiene un score prometedor, mañana seguir con este modelo
model_xgb.score(X_test, y_test)*100

78.93907563025209

In [53]:
#usando kfold y stratified kfold
model_xgb = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', max_depth=11, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, reg_lambda=0.1, n_jobs=1)

In [58]:
kfold = KFold(n_splits=6, random_state=100)
resultados = cross_val_score(model_xgb, X_train, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()*100))

Accuracy: 76.773785


In [59]:
kfold = StratifiedKFold(n_splits=8, random_state=134)
resultados = cross_val_score(model_xgb, X_train, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()*100))

Accuracy: 76.738903
